In [ ]:
# Notebook: exercicios_lab.ipynb

# Célula 2: Criar SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Lab Iceberg Exercises") \
    .config("spark.sql.catalog.lab", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.lab.type", "hive") \
    .config("spark.sql.catalog.lab.uri", "thrift://localhost:9083") \
    .enableHiveSupport() \
    .getOrCreate()

print("Spark session criada!")

# Célula 3: Exercício 2 - Criar DataFrame e salvar CSV
data = [(1, "Alice"), (2, "Bob"), (3, "Carol")]
columns = ["id", "nome"]

df = spark.createDataFrame(data, columns)
df.write.mode("overwrite").csv("hdfs://namenode:9000/data/ex1.csv", header=True)
print("CSV salvo no HDFS")

# Célula 4: Exercício 3 - Ler CSV do HDFS
df_read = spark.read.csv("hdfs://namenode:9000/data/ex1.csv", header=True, inferSchema=True)
df_read.show()

# Célula 5: Exercício 4 - Criar namespace Iceberg
spark.sql("CREATE NAMESPACE IF NOT EXISTS lab.db")

# Célula 6: Exercício 5 - Criar tabela Iceberg
spark.sql("CREATE TABLE lab.db.pessoas (id INT, nome STRING) USING ICEBERG")

# Célula 7: Exercício 6 - Inserir dados
spark.sql("INSERT INTO lab.db.pessoas VALUES (1,'Alice'), (2,'Bob'), (3,'Carol')")

# Célula 8: Exercício 7 - Ler tabela Iceberg
spark.sql("SELECT * FROM lab.db.pessoas").show()

# Célula 9: Exercício 8 - Contar registros
spark.sql("SELECT COUNT(*) AS total FROM lab.db.pessoas").show()

# Célula 10: Exercício 9 - Atualizar um registro
spark.sql("UPDATE lab.db.pessoas SET nome = 'Alice Silva' WHERE nome = 'Alice'")

# Célula 11: Exercício 10 - Deletar um registro
spark.sql("DELETE FROM lab.db.pessoas WHERE nome = 'Bob'")

# Célula 12: Exercício 11 - Criar tabela particionada
spark.sql("""
CREATE TABLE lab.db.vendas (
    id INT,
    valor DOUBLE,
    ano INT
) USING ICEBERG
PARTITIONED BY (ano)
""")

# Célula 13: Exercício 12 - Inserir dados particionados
spark.sql("""
INSERT INTO lab.db.vendas VALUES
(1, 100.0, 2022),
(2, 200.0, 2023),
(3, 300.0, 2023),
(4, 400.0, 2024)
""")

# Célula 14: Exercício 13 - Consultar apenas um particionamento
spark.sql("SELECT * FROM lab.db.vendas WHERE ano = 2023").show()

# Célula 15: Exercício 14 - Ver metadados da tabela
spark.sql("DESCRIBE HISTORY lab.db.vendas").show()
spark.sql("DESCRIBE DETAIL lab.db.vendas").show()

# Célula 16: Exercício 15 - Criar tabela Iceberg a partir de DataFrame
df2 = spark.createDataFrame([(10, "Produto A"), (20, "Produto B")], ["id", "nome"])
df2.writeTo("lab.db.tabela_df").createOrReplace()

# Célula 17: Exercício 16 - Converter tabela Parquet para Iceberg
# Criar tabela Parquet
spark.sql("""
CREATE TABLE lab.db.parquet_table_parquet (
    id INT,
    nome STRING
) USING PARQUET
""")
# Inserir dados na tabela Parquet
spark.sql("INSERT INTO lab.db.parquet_table_parquet VALUES (1,'Teste1'), (2,'Teste2')")
# Converter para Iceberg
spark.sql("ALTER TABLE lab.db.parquet_table_parquet SET TBLPROPERTIES ('format-version'='2')")

# Célula 18: Exercício 17 - Leitura incremental (Time Travel)
# Para garantir versão, fazemos uma atualização
spark.sql("UPDATE lab.db.vendas SET valor = valor + 10 WHERE ano = 2022")
spark.sql("SELECT * FROM lab.db.vendas VERSION AS OF 1").show()

# Célula 19: Exercício 18 - Exportar tabela Iceberg para CSV
df_vendas = spark.table("lab.db.vendas")
df_vendas.write.mode("overwrite").csv("hdfs://namenode:9000/export/vendas.csv", header=True)

# Célula 20: Exercício 19/20 - Dashboard / Trino (anotação)
"""
Exercício 19: Criar Dashboard no Superset
- Adicionar banco Trino
- Conectar ao catálogo Iceberg
- Criar visualização

Exercício 20: Ler tabela Iceberg via Trino
- SELECT * FROM iceberg.lab.db.pessoas;
"""
